In [2]:
import glob
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [3]:
path_fischer = 'chess-games-dataset/Data/CSV_FISCHER'
path_morphy = 'chess-games-dataset/Data/CSV_MORPHY'
path_capablanca = 'chess-games-dataset/Data/CSV_CAPABLANCA'

files_fischer = glob.glob(path_fischer + "/*.csv")
#files_morphy = glob.glob(path_morphy + "/*.csv")
#files_capablanca = glob.glob(path_capablanca + "/*.csv")

            
li = []

for filename in files_fischer:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

train = pd.concat(li, axis=0, ignore_index=True)

In [4]:
train = shuffle(train)

In [5]:
train.shape

(830376, 193)

In [6]:
train.head()

,a1,b1,c1,d1,e1,f1,g1,h1,a2,b2,...,to_h7,to_a8,to_b8,to_c8,to_d8,to_e8,to_f8,to_g8,to_h8,good_move
673497,R,None,B,Q,K,None,None,R,P,P,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
479029,None,None,None,None,None,None,K,None,P,P,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,False
61144,R,None,None,R,None,B,K,None,None,P,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
272184,R,None,None,None,None,None,None,K,P,P,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
452203,R,N,B,Q,K,None,None,R,P,P,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False


In [7]:
features = list(train.iloc[:, 0:192].columns)

In [8]:
X = train[features]
y = train['good_move']

In [9]:
categorical_columns = list(X.iloc[:, 0:63].columns)
numerical_columns = list(X.iloc[:, 64:192].columns)
feature_columns = []

for feature_name in categorical_columns:
  vocabulary = X[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))


for feature_name in numerical_columns:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype = tf.float32))

In [10]:
def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size = 32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

In [11]:
def split_into_batches(df, batch_size=100000):
  nb_rows = len(df.index)
  intervals = []
  
  for i in range(0, nb_rows + 1, batch_size):
    intervals.append(i)
  
  if(intervals[-1] != nb_rows):
    intervals.append(nb_rows)
  
  batches_X = []
  batches_y = []
  
  for i in range(0, len(intervals) - 1):
    batches_X.append(train.iloc[intervals[i]:intervals[i + 1], :][features])
    batches_y.append(train.iloc[intervals[i]:intervals[i + 1], :]['good_move'])

  return batches_X, batches_y

In [12]:
batches_X, batches_y = split_into_batches(train)

In [13]:
linear_est = tf.estimator.LinearClassifier(feature_columns = feature_columns, model_dir='.')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '.', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [14]:
input_functions = []
for df_X, df_y in zip(batches_X, batches_y):
  input_functions.append(make_input_fn(df_X, df_y))

In [15]:
len(input_functions)

9

In [16]:
# train the model on all the input functions
i = 1
for input_function in input_functions:
  print('<======================================== NEW BATCH ========================================>')
  print('Batch: ' + str(i))
  i = i + 1
  linear_est.train(input_function)
  

# save the model
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec(feature_columns))

estimator_base_path = '.'
estimator_path = linear_est.export_saved_model(estimator_base_path, serving_input_fn)

<======================================== NEW BATCH ========================================>
Batch: 1
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.9/site-packages/keras/engine/base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.


2021-09-25 11:11:13.594643: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into ./model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 26.4517
INFO:tensorflow:loss = 0.17929277, step = 100 (3.782 sec)
INFO:tensorflow:global_step/sec: 119.797
INFO:tensorflow:loss = 0.2634318, step = 200 (0.835 sec)
INFO:tensorflow:global_step/sec: 109.385
INFO:tensorflow:loss = 0.15754677, step = 300 (0.915 sec)
INFO:tensorflow:global_step/sec: 103.847
INFO:tensorflow:loss = 0.023987528, step = 400 (0.963 sec)
INFO:tensorflow:global_step/sec: 108.002
INFO:tensorflow:loss = 0.02850405, step = 500 (0.925 sec)
INFO:tensorflow:global_step/sec: 81.0304
INFO:tensorflow:loss = 0.02223361, step = 600 (1.234 sec)
INFO:tensorflow:global_step/sec: 101.611
INFO:tensorflow:loss = 0.22197

...

INFO:tensorflow:global_step/sec: 137.461
INFO:tensorflow:loss = 0.19788623, step = 227650 (0.727 sec)
INFO:tensorflow:global_step/sec: 139.141
INFO:tensorflow:loss = 0.026236102, step = 227750 (0.719 sec)
INFO:tensorflow:global_step/sec: 134.116
INFO:tensorflow:loss = 0.14091648, step = 227850 (0.746 sec)
INFO:tensorflow:global_step/sec: 139.498
INFO:tensorflow:loss = 0.27585924, step = 227950 (0.717 sec)
INFO:tensorflow:global_step/sec: 116.036
INFO:tensorflow:loss = 0.02795042, step = 228050 (0.862 sec)
INFO:tensorflow:global_step/sec: 89.9488
INFO:tensorflow:loss = 0.11803998, step = 228150 (1.112 sec)
INFO:tensorflow:global_step/sec: 111.972
INFO:tensorflow:loss = 0.12541132, step = 228250 (0.893 sec)
INFO:tensorflow:global_step/sec: 112.448
INFO:tensorflow:loss = 0.029361721, step = 228350 (0.889 sec)
INFO:tensorflow:global_step/sec: 107.506
INFO:tensorflow:loss = 0.02947751, step = 228450 (0.930 sec)
INFO:tensorflow:global_step/sec: 124.24
INFO:tensorflow:loss = 0.040433057, step

INFO:tensorflow:loss = 0.024585877, step = 235650 (0.692 sec)
INFO:tensorflow:global_step/sec: 145.044
INFO:tensorflow:loss = 0.028224371, step = 235750 (0.689 sec)
INFO:tensorflow:global_step/sec: 143.103
INFO:tensorflow:loss = 0.11983636, step = 235850 (0.699 sec)
INFO:tensorflow:global_step/sec: 138.266
INFO:tensorflow:loss = 0.028051969, step = 235950 (0.723 sec)
INFO:tensorflow:global_step/sec: 141.902
INFO:tensorflow:loss = 0.13427816, step = 236050 (0.705 sec)
INFO:tensorflow:global_step/sec: 142.079
INFO:tensorflow:loss = 0.27183273, step = 236150 (0.704 sec)
INFO:tensorflow:global_step/sec: 145.065
INFO:tensorflow:loss = 0.250996, step = 236250 (0.689 sec)
INFO:tensorflow:global_step/sec: 145.963
INFO:tensorflow:loss = 0.03423155, step = 236350 (0.685 sec)
INFO:tensorflow:global_step/sec: 144.33
INFO:tensorflow:loss = 0.11835167, step = 236450 (0.693 sec)
INFO:tensorflow:global_step/sec: 144.076
INFO:tensorflow:loss = 0.039633386, step = 236550 (0.694 sec)
INFO:tensorflow:glob

INFO:tensorflow:global_step/sec: 144.065
INFO:tensorflow:loss = 0.19962798, step = 243750 (0.694 sec)
INFO:tensorflow:global_step/sec: 148.462
INFO:tensorflow:loss = 0.2607339, step = 243850 (0.674 sec)
INFO:tensorflow:global_step/sec: 147.579
INFO:tensorflow:loss = 0.04177268, step = 243950 (0.678 sec)
INFO:tensorflow:global_step/sec: 147.472
INFO:tensorflow:loss = 0.028834552, step = 244050 (0.678 sec)
INFO:tensorflow:global_step/sec: 145.654
INFO:tensorflow:loss = 0.22579007, step = 244150 (0.686 sec)
INFO:tensorflow:global_step/sec: 145.221
INFO:tensorflow:loss = 0.1903725, step = 244250 (0.689 sec)
INFO:tensorflow:global_step/sec: 144.501
INFO:tensorflow:loss = 0.25255644, step = 244350 (0.692 sec)
INFO:tensorflow:global_step/sec: 140.181
INFO:tensorflow:loss = 0.102994315, step = 244450 (0.713 sec)
INFO:tensorflow:global_step/sec: 141.677
INFO:tensorflow:loss = 0.22005892, step = 244550 (0.706 sec)
INFO:tensorflow:global_step/sec: 138.493
INFO:tensorflow:loss = 0.03549987, step =

INFO:tensorflow:global_step/sec: 119.393
INFO:tensorflow:loss = 0.20852807, step = 250900 (0.838 sec)
INFO:tensorflow:global_step/sec: 116.384
INFO:tensorflow:loss = 0.2747483, step = 251000 (0.860 sec)
INFO:tensorflow:global_step/sec: 111.539
INFO:tensorflow:loss = 0.033002116, step = 251100 (0.897 sec)
INFO:tensorflow:global_step/sec: 120.737
INFO:tensorflow:loss = 0.13685983, step = 251200 (0.827 sec)
INFO:tensorflow:global_step/sec: 130.131
INFO:tensorflow:loss = 0.124862775, step = 251300 (0.768 sec)
INFO:tensorflow:global_step/sec: 131.362
INFO:tensorflow:loss = 0.15189993, step = 251400 (0.761 sec)
INFO:tensorflow:global_step/sec: 135.296
INFO:tensorflow:loss = 0.029115438, step = 251500 (0.739 sec)
INFO:tensorflow:global_step/sec: 131.495
INFO:tensorflow:loss = 0.13146542, step = 251600 (0.761 sec)
INFO:tensorflow:global_step/sec: 132.121
INFO:tensorflow:loss = 0.16371371, step = 251700 (0.757 sec)
INFO:tensorflow:global_step/sec: 134.16
INFO:tensorflow:loss = 0.124550804, step

INFO:tensorflow:loss = 0.21484335, step = 258900 (0.726 sec)
INFO:tensorflow:global_step/sec: 136.396
INFO:tensorflow:loss = 0.028456893, step = 259000 (0.733 sec)
INFO:tensorflow:global_step/sec: 135.094
INFO:tensorflow:loss = 0.12542783, step = 259100 (0.740 sec)
INFO:tensorflow:global_step/sec: 135.151
INFO:tensorflow:loss = 0.035869647, step = 259200 (0.740 sec)
INFO:tensorflow:global_step/sec: 135.776
INFO:tensorflow:loss = 0.03283293, step = 259300 (0.737 sec)
INFO:tensorflow:global_step/sec: 135.801
INFO:tensorflow:loss = 0.18335727, step = 259400 (0.736 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 259500...
INFO:tensorflow:Saving checkpoints for 259500 into ./model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 259500...
INFO:tensorflow:Loss for final step: 0.025683997.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 c